скачиваем wget и импортируем нужные библиотеки

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=7270a49a2d867631555166cc8e4899d71d2fde722bd1a52d8658dcceaada2e16
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import wget

In [ ]:
# сохраняем ссылку на файл в переменную
url = 'https://www.netoptik.ru/yandexmarket.yml'

# загружаем данные
data = requests.get(url)

# проверяем что всё нормально
data.status_code

200

In [ ]:
# парсим данные
soup = BeautifulSoup(data.text, 'xml')

In [ ]:
# сохраняем в переменную все товары
offers = soup.findAll('offer')

In [ ]:
# смотрим как выглядит один товар
offers[2]

<offer id="11449">
<categoryId>18</categoryId>
<name>Оправа Boss Hugo Boss, BOSS 1312, 003</name>
<url>https://www.netoptik.ru/frames/men-frames/oprava-boss-hugo-boss-boss1312-003</url>
<price>13500</price>
<currencyId>RUB</currencyId>
<description>&amp;amp;lt;p&amp;amp;gt;&amp;amp;lt;br&amp;amp;gt;&amp;amp;lt;/p&amp;amp;gt;</description>
<vendor>BOSS HUGO BOSS</vendor>
<model>Оправа Boss Hugo Boss, BOSS 1312, 003</model>
<picture>https://www.netoptik.ru/image/cache/catalog/Boss%20Hugo%20Boss/boss-hugo-boss-boss1312-003-s2-1%20(2)-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/catalog/Boss%20Hugo%20Boss/boss-hugo-boss-boss1312-003-s2-2%20(1)-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/catalog/Boss%20Hugo%20Boss/boss-hugo-boss-boss1312-003-s2-3%20(3)-600x600.jpg</picture>
<param name="Категория">Оправы для очков</param>
<param name="Ширина оправы (мм)">143</param>
<param name="Ширина линзы (мм)">57</param>
<param name="Высота линзы (мм)">42</pa

так можно получить ссылки на изображения одного товара

In [ ]:
# для каждого тега 'picture'
for i in offers[0].findAll('picture'):

  # выводим содержимое самого тега
  print(i.get_text())

https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s213-600x600.jpg
https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s231-600x600.jpg
https://www.netoptik.ru/image/cache/data/med_opravi/BALENCIAGA/balenciagabal01088o0s222-600x600.jpg


вспомогательная функция, чтобы отделить как можно больше другого товара от очков

In [ ]:
def glasses_or_not(text):

  # список слов исключений
  exception_words = ['оправ', 'линз', 'клипоны', 'лентикуляры',
                     'однодневные', 'маски', 'двухнедельные',
                     'ежемесячные', 'фильтры', 'пенсне']

  # для каждого слова в списке
  for word in exception_words:

    # если слово есть в названии категории
    if word in text:

      # возвращаем False
      return False

  # если прошли по списку и не нашли исключений то возвращаем True
  return True

в этой ячейке мы сохраняем ссылку на изображение и материал в два отдельных списка

In [ ]:
# список ссылок
links_list = []

# список материалов
material_list = []

# для каждого товара
for offer in offers:

  # выделяем категорию
  category = offer.find('param', {'name': 'Категория'})

  # выделяем материал
  material = offer.find('param', {'name': 'Материал '})

  # если категория существует и в её тексте нет слов исключений
  if category is not None and glasses_or_not(category.get_text()):

    # если материал существует(то есть указан)
    if material is not None:

      # в товаре выделяем все сслыки на мзображения
      for link in offer.findAll('picture'):

        # в список ссылок добавляем ссылку
        links_list.append(link.get_text())

        # в список материалов добавляем материал в нижнем регистре
        material_list.append(material.get_text().lower())

проверим что в списке ссылок нет ничего лишнего

In [ ]:
print(len(links_list), len(material_list))

for i in range(len(links_list) - 1):
  if not links_list[i].startswith('https://'):
    links_list.pop(i)
    material_list.pop(i)

print(len(links_list), len(material_list))

2857 2857
2856 2856


In [ ]:
material_set = set(material_list)
print(material_set)
print(len(material_set))

{'комбинированный', 'трайвекс', 'титан', 'силикон', 'пластик, ударопрочный', 'металл', 'поликарбонат', 'silflex', 'пластик'}
9


In [ ]:
# задаём базовый путь, где будут храниться изображения
base_path = '/content/drive/MyDrive/Projects/Netoptic/photos'

In [ ]:
for i in range(len(links_list)):

  os.makedirs(os.path.join(base_path, material_list[i]), exist_ok=True)

  img_name = links_list[i].split('/')[-1]

  img = requests.get(links_list[i])

  save_path = os.path.join(base_path, material_list[i], img_name)

  with open(save_path, 'wb') as out:
    out.write(img.content)